In [ ]:
def modeloPolinomio(self,listZ,b):
        y=np.array([(b[0]*zi**2+b[1]*zi+b[2]).sum() for zi in listZ])

In [ ]:
import numpy as np
import cv2
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
from calibracion import graficar
from medicion import *
import matplotlib.pyplot as plt
import numpy as np
class volumen(imagen):######################## volumen en cm3
    def __init__(self,path_configuracion,nombre_configuracion):
      
        imagen.__init__(self,path_configuracion,nombre_configuracion,pendientes2=True)
        self.beta=np.array([ 0.12616642,  0.06137007, -0.00176466])# coeficientes de la calibración de area por optimización
        self.maxZ=25000
        self.minZ=3000
    def modeloArea(self,imgz):
        area=self.beta[0]*imgz**2+self.beta[1]*imgz+self.beta[2]
        return area
 
    def preprocesar_imgs(self,img_rgb,img_depth,f_segmentador,negro=True):
#         buffer=img_depth.copy()
        #quitar distorsion z
        img_depth=self.recuperar(img_depth)
        img_depth=self.distorsion_z2(img_depth,filtrar=False)
#         graficar([img_depth,buffer])
        #quitar distorsion uv
        img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
        # cambiar de dominio rgb a depth
        img2_depth=self.rgb2_depth(img_rgb)
        # encontrar mascara
    
        mascara,_=f_segmentador(img2_depth)
        return img_depth,mascara
    def calcular_volumen(self,img,mascara,img_ref):
      
        img_z=img.reshape(-1)
        mascara=mascara.reshape(-1)
        img_ref=img_ref.reshape(-1)
        # necesario para compatibilidad de dimensiones
        # extraer los pixeles del objetivo
        index=np.where(mascara==1)
        img_z=img_z[index]
        img_ref=img_ref[index]
        
        
#         graficar([img_z,img_ref],1,2,["img x mascara","ref"])
        znorm=self.normalizarz(img_z)
        img_area=self.modeloArea(znorm)
        print(img_area.sum(),"AREA TOTAL")
        
        img_altura=img_ref-img_z
        img_altura=img_altura/100 # pasar a cm
        print(img_altura.mean(),"ALTURA MEDIA")
        img_volumen=img_area*img_altura
        volumen=np.sum(img_volumen)
        print(volumen, "Volumen estimado")
        return volumen
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e
    def mejorar_referencia(self,img_ref):

#         img_ref[img_ref==0]=img_ref.mean()
        img_ref=self.distorsion_z(img_ref,1)

        return img_ref
    def abrir_img_ref(self,path):
        names = glob.glob("{}depth_*.png".format(path))
        ret=True if len(names)>0 else False
        imgRef=0
        for name in names:
            img=cv2.imread(name,cv2.IMREAD_UNCHANGED)[:,:,0]
            img=self.recuperar(img)
            img=self.distorsion_z2(img,filtrar=False)
            imgRef+=img
        imgRef/=len(names)
        return ret,imgRef
    def normalizarz(self, z):
        z=(z-self.minZ)/(self.maxZ-self.minZ)
        return z

In [ ]:
from tabulate import tabulate
from skimage import morphology
import cv2
import os
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion")
from calibracion import *


def medir(path_config,list_config,path_img,valor_real,titulo="Error"):
    nxpaso=4
    list_titulos=["error porcentual"]
    list_tablas=[]
    ejex=[]
    ejey=[]
    for config in list_config:
        print(config)
        a=volumen(path_config,config)
     
            # abrir el par de imagenes
        list_medidas=[]
        list_error=[]
        list_media=[]
        ret,imgRef=a.abrir_img_ref("{}fondo/".format(path_img))
        print(imgRef.dtype)
        plt.imshow(imgRef)
        plt.show()
#         graficar([imgRef],1,
        
        for i in range(1,1000):
            
            ret,img_rgb,img_depth=a.abrir_par_img(path=path_img,
                                            indicador_rgb="gray",
                                            indicador_depth="depth",
                                            nombre=i
                                           )
            if ret:
                depth=img_depth.copy()
                
                
       
                    
                #segmentar
                img_depth,mascara=a.preprocesar_imgs(img_rgb,img_depth,segmetador)
                #calcular
                print(img_depth.dtype,mascara.dtype)
                mod2=cv2.addWeighted(img_depth, 0.5, 3000*mascara.astype(np.float64), 0.5, 0)
                
                graficar([depth,img_depth,mod2],1,3)
                print("valor medio profundidad mm: ",img_depth.mean()/10)
                valor_calculado=a.calcular_volumen(img_depth,mascara,imgRef)
                
                error=a.calcular_error(valor_real,valor_calculado)
                print("error",error)
                list_medidas.append(valor_calculado)
                list_error.append(error)
                list_media.append(imgRef.max())
                print("valor medio profundidad mm: ",img_depth.mean()/10)
    #             print("valor real mm²: {}".format(valor_real))
    #             print("area calculada mm²:{}".format(area_calculada))
    #             print("porcentaje de error:{}".format(error))

    #     print(valor_real)
        tabla=zip(list_medidas,list_error,list_media)
        ejex.append(list_media)
        ejey.append(np.absolute(list_error))
        list_titulos.append(config)
#         print("valor real:{}".format(valor_real))
#         print(tabulate(tabla,headers=["v calculado","error","distancia media"]))
        print(" Error absoluto medio: ", np.absolute(np.array(list_error)).mean())
#     plot([list_media],[np.absolute(list_error)],1,1,[titulo],[["Profundidad","Error absoluto %"]],tipo=1)
    plt.hist(list_error) 
    return list_error,list_media
    
def segmetador(img):
    n=200
    kernel = np.ones((n,n),np.uint8)
    img_gray=cv2.cvtColor(img.copy(), cv2.COLOR_RGB2BGR)
    
    img_gray=cv2.cvtColor(img.copy(), cv2.COLOR_RGB2GRAY)
    #img_= cv2.medianBlur(img_gray,15)

    criterio= cv2.THRESH_BINARY+cv2.THRESH_OTSU
#     img_gray = cv2.morphologyEx(img_gray, cv2.MORPH_BLACKHAT, kernel)#MORPH_BLACKHAT
#     graficar([img_gray],1,1)
    thr,_ =cv2.threshold(img_gray,0,1,criterio)
#     thr,_ =cv2.threshold(img_gray[img_gray<=thr],0,1,criterio)
#     thr,_ =cv2.threshold(img_gray[img_gray<=thr],0,1,criterio)

    mascara=img_gray.copy()

    mascara[mascara<=thr]=1
    mascara[mascara!=1]=0
    mascara=mascara.astype(dtype=bool)
#     mascara.dtype=bool## remove_small solo funciona con variables binarias
    mascara=morphology.remove_small_objects(mascara,500,connectivity=50)
    mascara.dtype=np.uint8
    img_mascara=cv2.bitwise_and(img,img,mask=mascara)
    mod=img_gray.copy()
    mod[mod>thr]=0
    graficar([mod,img_mascara,img],1,3)
    return mascara,img_mascara


# abrir img
img=cv2.imread("/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/prueba/volumen1/gray_143.png")
mascara,img_mascara=segmetador(img)
# print("humbral: {}".format(thr))

graficar([img,mascara,img_mascara],1,3,["original","mascara","original X mascara"])

In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/volumen1/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=8*10*21.5*6
volumen1,list_media_cubo1=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
v1=np.array(volumen1).mean()
z1=np.array(list_media_cubo1).mean()



In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/volumen2/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=8*10*21.5*6
volumen2,list_media_cubo2=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
v2=np.array(volumen2).mean()
z2=np.array(list_media_cubo2).mean()


In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/volumen3/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=8*10*21.5*6
volumen3,list_media_cubo3=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
v3=np.array(volumen3).mean()
z3=np.array(list_media_cubo3).mean()


In [ ]:
10*21.5*7

In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/volumen4/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=8*10*21.5*7
volumen4,list_media_cubo4=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
v4=np.array(volumen4).mean()
z4=np.array(list_media_cubo4).mean()


In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/volumen5/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=8*10*21.5*7
volumen5,list_media_cubo5=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
v5=np.array(volumen5).mean()
z5=np.array(list_media_cubo5).mean()


In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/volumen6/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=8*10*21.5*7
volumen6,list_media_cubo6=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
v6=np.array(volumen6).mean()
z6=np.array(list_media_cubo6).mean()


In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/volumen7/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=8*10*21.5*7
volumen7,list_media_cubo7=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
v7=np.array(volumen7).mean()
z7=np.array(list_media_cubo7).mean()


In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/volumen8/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=8*10*21.5*7
volumen8,list_media_cubo8=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
v8=np.array(volumen8).mean()
z8=np.array(list_media_cubo8).mean()


In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/volumen9/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=8*10*21.5*7
volumen9,list_media_cubo9=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
v9=np.array(volumen9).mean()
z9=np.array(list_media_cubo9).mean()


In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/volumen10/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=8*10*21.5*7
volumen10,list_media_cubo10=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
v10=np.array(volumen10).mean()
z10=np.array(list_media_cubo10).mean()


In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/volumen11/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=8*10*21.5*7
volumen11,list_media_cubo11=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
v11=np.array(volumen11).mean()
z11=np.array(list_media_cubo11).mean()


In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/volumen12/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=8*10*21.5*7
volumen12,list_media_cubo12=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
v12=np.array(volumen12).mean()
z12=np.array(list_media_cubo12).mean()


In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/volumen13/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=8*10*21.5*7
volumen13,list_media_cubo13=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
v13=np.array(volumen13).mean()
z13=np.array(list_media_cubo13).mean()


In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/volumen14/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=8*10*21.5*7
volumen14,list_media_cubo14=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
v14=np.array(volumen14).mean()
z14=np.array(list_media_cubo14).mean()


In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/volumen15/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=8*10*21.5*7
volumen15,list_media_cubo15=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
v15=np.array(volumen15).mean()
z15=np.array(list_media_cubo15).mean()


In [ ]:
lista=np.concatenate([volumen6,volumen7,volumen8,volumen9,volumen10,volumen11])
plt.hist(lista)
print(lista.mean())

In [ ]:
error=[v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15]
profundidad=[z1,z2,z3,z4,z5,z6,z7,z8,z9,z10,z11,z12,z13,z14,z15]
plt.plot(profundidad,error)

In [ ]:
error=[v5,v6,v7,v8,v9,v10,v11,v12]
profundidad=[z5,z6,z7,z8,z9,z10,z11,z12]
plt.plot(profundidad,error)

In [ ]:
error=np.array(error)+16
plt.plot(profundidad,error)

In [ ]:
profundidad

# medición de un solo objetivo

In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/prueba/volumen1/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=20*20*20
volumen1,list_media_cubo1=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
v1=np.array(volumen1).mean()
z1=np.array(list_media_cubo1).mean()


In [ ]:
(400-545.4203229231593)/400

In [ ]:
(20-17)/20

In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/cajas/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=45.5*55.5*36
volumen1,list_media_cubo1=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
v1=np.array(volumen1).mean()
z1=np.array(list_media_cubo1).mean()


In [ ]:
(45.5*55.5-2517)/45.5*55.5

In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/cajas1/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=66*30*30.5
volumen1,list_media_cubo1=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
v1=np.array(volumen1).mean()
z1=np.array(list_media_cubo1).mean()


In [ ]:
66*30

In [ ]:
(30.5-29.89)/30.5

In [ ]:
# medición de cubo
%matplotlib inline

path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/cajas2/"
path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
list_config=["distorsionz"]
valor_real=66*30*30.5+45.5*55.5*36
volumen1,list_media_cubo1=medir(path_config,list_config,path_img,valor_real,titulo="cubo")
v1=np.array(volumen1).mean()
z1=np.array(list_media_cubo1).mean()